In [1]:
import os, sys, itertools, glob, re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from functools import partial


In [2]:
possible_models = {'esvr': 'Epsilon-Support Vector Regression',
                   'erf': 'Extremely Random Forest',
                   'lr': 'LinearRegression',
                   'knn': 'Nearest-Neighbor',
                   'dnn': 'RASPDeep',
                   'rf': 'Random Forest',
                   'svr': 'Support Vector Regression'}
modelstr = " ".join(possible_models)
modelstr

'esvr erf lr knn dnn rf svr'

In [3]:
n_repls = 10
n_cores = 8

weight_dir = "../data/raspdata_200503_180444/"


work_path = "../enrich_results"
os.makedirs(work_path, exist_ok=True)


dud_path = "../Manuscript_RASPD+_SI_DUD30/DUD-Parameters/parameters"

In [4]:
dud_names = os.listdir(dud_path)
dud_names

['ACE',
 'AChE',
 'ADA',
 'AMPC',
 'AR',
 'CDK2',
 'COX2',
 'DHFR-MTX',
 'EGFR',
 'ER',
 'ER-agonist',
 'FGFr1',
 'FXA',
 'GPB',
 'GR',
 'HIVPR',
 'HIVRT',
 'HSP90',
 'InhA',
 'MR',
 'NA',
 'P38',
 'PDE5',
 'PPARg',
 'PR',
 'SRC',
 'Thrombin',
 'Thymidinekinase',
 'Trypsin',
 'VEGFr2']

In [5]:
parameter_files = {}
id_files = {}
subdirs = {}
for name in dud_names:
    glob_pattern = os.path.join(dud_path, name, "run", "*union", "tmpdir", "PARAMETERS_MILLION.txt")
    parameter_file = glob.glob(glob_pattern)[0] 
    parameter_files[name] = parameter_file
    assert os.path.exists(parameter_file)
    id_file = os.path.join(dud_path, name, "run", "list_customized")
    id_files[name] = id_file
    assert os.path.exists(id_file)
    assert pd.read_csv(parameter_file).shape[0] ==  pd.read_csv(id_file, header=None).shape[0]

    subdirs[name] = {}
    for i in range(n_repls):
        tmp_out = os.path.join(work_path, name, str(i))
        subdirs[name][i] = tmp_out
        #os.makedirs(tmp_out, exist_ok=True)
        #os.system(f"python infer.py -d {tmp_out} -w {weight_dir} -i {parameter_file} -m {modelstr} -s {i} -c {n_cores}")


In [6]:
def compute_ef_single(df, fractions=[0.01, 0.05, 0.1], ascending=True):
    actives = df['active'].sum()
    decoys = (~df['active']).sum()
    total = df.shape[0]
    assert total == actives + decoys
    efs = {}
    for frac in fractions:
        portion = int(total * frac)
        recovered = df.sort_values('value', ascending=ascending).iloc[:portion]['active'].sum()
        ef = (recovered / portion) / (actives / total)
        efs[str(frac)] = ef
    return {"actives": actives, "decoys": decoys, **efs}

In [7]:
def compute_ef_union(dfs, fractions=[0.01, 0.05, 0.1], ascending=True, active_str="AC_"):
    dfs = [df.sort_values('value', ascending=ascending) for df in dfs]
    actives = dfs[0]['active'].sum()
    decoys = (~dfs[0]['active']).sum()
    total = dfs[0].shape[0]
    assert total == actives + decoys
    efs = {}
    for frac in fractions:
        portion = int(total * frac)
        recover = set()
        for cands in zip(*[df['id'] for df in dfs]):
            if len(recover.union(cands)) > portion:
                break
            recover.update(cands)
        assert len(recover) <= portion
        assert portion - len(recover) < 7
        uniqued = pd.Series(list(recover))
        recovery_rate = uniqued.str.match(active_str).mean()
        efs[str(frac)] = recovery_rate / (actives / total)
    return efs

In [8]:
def mean_result_df(dfs):
    id_frame = dfs[0][['id', 'active']]
    mean_value = pd.concat([df['value'] for df in dfs], axis=1).mean(axis=1)
    mean_value = pd.Series(mean_value, name='value')
    return id_frame.join(mean_value)

In [18]:
def compute_ef_table(output_list, id_list):
    res = {}
    efs = {}
    for name in dud_names:
        res[name] = {}
        efs[name] = {}
        with open(id_list[name], 'r') as id_file:
            ids = pd.Series(list(id_file), name='id')
        for repl in range(n_repls):
            res[name][repl] = {}
            efs[name][repl] = {}
            for model in possible_models:
                res_file = os.path.join(output_list[name][repl], f"{model}.out")
                df = (pd.read_csv(res_file, delim_whitespace=True, header=None, names=['value', 'std'])
                        .drop(columns=['std']))
                assert df.shape[0] == ids.shape[0]
                dat = df.join(ids)
                dat['active'] = dat['id'].str.match('AC_')
                dat = dat.sort_values('value', ascending=True)
                res[name][repl][model] = dat
                efs[name][repl][model] = compute_ef_single(dat)
            efs[name][repl] = pd.DataFrame.from_records(efs[name][repl])
            mean_ensemble = compute_ef_single(mean_result_df(list(res[name][repl].values())))
            mean_ens_dknn = compute_ef_single(mean_result_df([val for key, val in res[name][repl].items() if key != "knn"]))
            union = compute_ef_union(res[name][repl].values())
            union_dknn = compute_ef_union(val for key, val in res[name][repl].items() if key != "knn")
            efs[name][repl] = efs[name][repl].join(
                pd.DataFrame({'mean_ensemble': pd.Series(mean_ensemble),
                              'mean_ens_dknn': pd.Series(mean_ens_dknn),
                              'union': pd.Series(union),
                              'union_dknn': pd.Series(union_dknn),
                             }))
        efs[name] = pd.concat(efs[name], names=['replicate'])


    efs = pd.concat(efs, names=['system'])
    return efs, res

In [19]:
enrichments, details = compute_ef_table(subdirs, id_files)


In [20]:
value_order = ['erf', 'rf', 'dnn', 'knn', 'svr', 'esvr', 'lr', 
               'union', 'union_dknn', 'mean_ensemble', 'mean_ens_dknn']

enrichments = enrichments[value_order]

enrichments.index.rename('level', level=2, inplace=True)
enrichments.columns.rename('method', inplace=True)


In [21]:
def texify_val_error(values, nan_text="NA", sig_digits=2, force_sig=False):
    nan_check = pd.isna(values)
    if nan_check[0]:
        return nan_text
    elif nan_check[1] or values[1] == 0:
        return "{{:.{}f}}".format(sig_digits).format(values[0])
    else:
        error = "{:.2g}".format(values[1])
        split_error = error.split(".")
        if len(split_error) == 2:
            sig = len(split_error[1])
            if force_sig:
                sig = min(sig, sig_digits)
            value = '{{:.{prec}f}}'.format(prec=sig).format(values[0])
            error = '{{:.{prec}f}}'.format(prec=sig).format(values[1])
        else:
            error = str(round(values[1]))
            value = str(round(values[0]))
        return "\\num{{{} \\pm {}}}".format(value, error)


def texify_df_groupby(df, axis=1, stats=('mean', 'std'), force_sig=False):
    trans_func=None
    if force_sig:
        trans_func = functools.partial(texify_val_error, force_sig=True)
    else:
        trans_func = texify_val_error
    result = (df.agg([*stats])
         .groupby(axis=axis, level=0).transform(trans_func)
         .xs("mean", axis=axis, level=1)
         )
    return result

In [22]:
enrichments.stack().groupby(level=['level', 'method']).agg(['mean', 'std']).unstack().T

level                   0.01      0.05       0.1     actives       decoys
     method                                                              
mean erf            4.974497  1.983633  1.323233  116.466667  3599.133333
     rf             5.405880  2.051063  1.387311  116.466667  3599.133333
     dnn            4.022107  1.570880  1.207944  116.466667  3599.133333
     knn            1.477251  1.037577  0.874338  116.466667  3599.133333
     svr            6.062478  1.917705  1.327958  116.466667  3599.133333
     esvr           4.434033  1.740438  1.262252  116.466667  3599.133333
     lr             6.213404  1.905964  1.333081  116.466667  3599.133333
     union          7.326559  2.580138  1.684054         NaN          NaN
     union_dknn     7.492562  2.590600  1.667372         NaN          NaN
     mean_ensemble  5.403853  1.775772  1.232674  116.466667  3599.133333
     mean_ens_dknn  6.555140  1.984227  1.305951  116.466667  3599.133333
std  erf            5.193826  1.695557  0.983948  131.373942  3572.063536
     rf             5.271920  1.636794  0.945450  131.373942  3572.063536
     dnn            4.217783  1.296824  0.864854  131.373942  3572.063536
     knn            3.132364  1.395334  0.901992  131.373942  3572.063536
     svr            5.710314  1.610062  0.932510  131.373942  3572.063536
     esvr           4.197524  1.355666  0.868642  131.373942  3572.063536
     lr             5.936847  1.659914  1.039817  131.373942  3572.063536
     union          5.858357  1.687947  0.943907         NaN          NaN
     union_dknn     5.646276  1.741903  0.959126         NaN          NaN
     mean_ensemble  4.769066  1.467405  0.866958  131.373942  3572.063536
     mean_ens_dknn  5.475333  1.539119  0.853529  131.373942  3572.063536

In [23]:
mean_state = enrichments.stack().groupby(level=['level', 'method']).agg(['mean', 'std']).unstack().T.iloc[0,-2:]
ratio = mean_state.loc['decoys'] / mean_state.loc['actives']
print(ratio)

30.90269032627361


In [24]:
enrichments

method                            erf           rf          dnn          knn  \
system replicate level                                                         
ACE    0         0.01       10.613426     4.245370    12.736111     0.000000   
                 0.05        4.198718     4.198718     5.038462     0.000000   
                 0.1         2.714253     2.923042     2.923042     0.000000   
                 actives    48.000000    48.000000    48.000000    48.000000   
                 decoys   1786.000000  1786.000000  1786.000000  1786.000000   
       1         0.01       12.736111     4.245370    12.736111     0.000000   
                 0.05        4.618590     4.618590     5.458333     0.000000   
                 0.1         2.714253     2.714253     3.340619     0.000000   
                 actives    48.000000    48.000000    48.000000    48.000000   
                 decoys   1786.000000  1786.000000  1786.000000  1786.000000   
       2         0.01       16.981481    10.613426    14.858796     0.000000   
                 0.05        4.618590     4.618590     5.878205     0.000000   
                 0.1         3.340619     2.923042     3.340619     0.000000   
                 actives    48.000000    48.000000    48.000000    48.000000   
                 decoys   1786.000000  1786.000000  1786.000000  1786.000000   
       3         0.01       16.981481    16.981481     6.368056     0.000000   
                 0.05        5.458333     5.878205     4.618590     0.419872   
                 0.1         3.131831     3.340619     3.549408     0.835155   
                 actives    48.000000    48.000000    48.000000    48.000000   
                 decoys   1786.000000  1786.000000  1786.000000  1786.000000   
       4         0.01       16.981481    12.736111     6.368056     0.000000   
                 0.05        5.458333     5.458333     4.198718     0.000000   
                 0.1         3.340619     2.923042     2.505464     0.208789   
                 actives    48.000000    48.000000    48.000000    48.000000   
                 decoys   1786.000000  1786.000000  1786.000000  1786.000000   
       5         0.01       21.226852     0.000000    10.613426     0.000000   
                 0.05        6.298077     3.358974     4.618590     0.000000   
                 0.1         3.340619     2.714253     2.923042     0.208789   
                 actives    48.000000    48.000000    48.000000    48.000000   
                 decoys   1786.000000  1786.000000  1786.000000  1786.000000   
...                               ...          ...          ...          ...   
VEGFr2 4         0.01        9.282132     8.121865     6.961599     1.160266   
                 0.05        2.728194     2.500845     1.364097     1.136748   
                 0.1         1.705121     1.705121     1.023073     1.364097   
                 actives    88.000000    88.000000    88.000000    88.000000   
                 decoys   2873.000000  2873.000000  2873.000000  2873.000000   
       5         0.01        4.641066     2.320533     8.121865     1.160266   
                 0.05        2.500845     0.909398     2.273495     1.818796   
                 0.1         1.818796     1.023073     1.136748     1.477772   
                 actives    88.000000    88.000000    88.000000    88.000000   
                 decoys   2873.000000  2873.000000  2873.000000  2873.000000   
       6         0.01       12.762931    10.442398     5.801332     2.320533   
                 0.05        3.864942     3.864942     1.364097     1.818796   
                 0.1         2.159820     2.387170     1.023073     1.818796   
                 actives    88.000000    88.000000    88.000000    88.000000   
                 decoys   2873.000000  2873.000000  2873.000000  2873.000000   
       7         0.01        5.801332     5.801332     5.801332     3.480799   
                 0.05        2.273495     2.046146     1.591447     2.500845   


In [25]:
enrichments.drop(columns=['mean_ens_dknn']).drop(index=['actives', 'decoys'], level='level').groupby(level=['system', 'level']).agg('mean')

method                       erf         rf        dnn        knn        svr  \
system          level                                                          
ACE             0.01   13.797454   7.641667   8.490741   0.000000   6.155787   
                0.05    4.996474   4.408654   4.408654   0.041987   2.939103   
                0.1     3.027436   2.818648   3.090073   0.167031   1.503279   
AChE            0.01    5.901419   7.770202   4.032636   0.885213   6.786632   
                0.05    3.134942   3.349130   1.947169   1.810867   2.648149   
                0.1     2.194730   2.039351   1.767438   1.514947   2.136463   
ADA             0.01    3.005413   4.917949   3.551852   0.273219   8.196581   
                0.05    1.464921   1.569558   2.040426   1.046372   1.569558   
                0.1     0.776518   0.776518   1.553036   0.724750   1.216545   
AMPC            0.01    1.087075   2.717687   0.000000   0.000000   0.000000   
                0.05    1.560928   1.853602   0.487790   0.682906   0.000000   
                0.1     0.963231   1.541169   1.685654   0.674262   0.288969   
AR              0.01   10.643911   8.869926   5.195242   0.633566   9.250065   
                0.05    4.257564   3.725369   1.444531   0.228084   2.939747   
                0.1     2.901733   3.167831   1.127748   0.291440   1.735971   
CDK2            0.01    3.613946   4.047619   8.818027   0.000000   6.938776   
                0.05    1.431941   1.804245   2.720687   0.601415   2.806604   
                0.1     1.145553   1.260108   1.632412   0.587096   2.104953   
COX2            0.01    1.724504   2.622191   2.622191   0.000000   2.385957   
                0.05    0.646322   0.820876   1.424739   0.023588   1.037889   
                0.1     0.417515   0.528380   1.287926   0.037741   0.990713   
DHFR-MTX        0.01    0.833712   0.441377   0.000000   0.637545   1.569341   
                0.05    0.533317   0.621387   0.132106   1.526557   0.738815   
                0.1     0.445246   0.467264   0.200605   1.472736   0.773064   
EGFR            0.01   13.739401  11.971385  10.011656   1.150275  13.164263   
                0.05    4.328444   3.719224   3.259114   0.813713   4.145252   
                0.1     2.575889   2.282110   2.116061   0.989908   2.667429   
ER              0.01    3.509524   0.539927   0.000000   0.269963   0.000000   
                0.05    2.122726   1.760309   0.000000   0.880155   0.000000   
                0.1     1.362672   1.336961   0.000000   1.079853   0.000000   
...                          ...        ...        ...        ...        ...   
NA              0.01   17.610526  16.562281   5.241228   2.096491   8.385965   
                0.05    5.912105   4.989649   2.096491   2.725439   2.096491   
                0.1     3.524520   2.836300   1.251309   2.398342   1.251309   
P38             0.01    1.045705   1.913417   1.268195   0.711969   3.493099   
                0.05    0.670663   1.036880   0.803031   0.414752   1.544290   
                0.1     0.716236   0.866095   0.698606   0.590619   1.154793   
PDE5            0.01    0.118372   0.118372   3.787907   0.000000   3.551163   
                0.05    0.351600   0.421920   1.453281   0.093760   1.640801   
                0.1     0.419842   0.851346   1.061267   0.081636   1.306175   
PPARg           0.01    0.241366   0.000000   0.000000   1.568880   0.000000   
                0.05    0.870588   0.352941   0.094118   1.129412   0.000000   
                0.1     0.729412   0.423529   0.141176   1.223529   0.164706   
PR              0.01    3.903704   6.245926   3.903704   1.171111   7.807407   
                0.05    1.651567   1.501425   1.050997   0.900855   1.651567   
                0.1     1.189700   1.003810   0.743563   0.557672   1.040988   
SRC             0.01   10.468746  11.753255   3.403948   2.183665  19.845660   
                0.05    5.149712   5.619021   2.004075   1.433294   6.215170   


In [26]:
method_renamer = {"erf": "eRF", 
                  "rf": "RF", 
                  "dnn": "DNN",
                  "svr": "lSVR", 
                  "esvr": "SVR", 
                  "knn": "kNN",  
                  "lr": "LR",
                  "mean_ensemble": "Mean",
                  "union_out": "Union",
                  "union_dknn": "Union w/o kNN"
                 }

In [27]:
system_rename = {'ACE': "ACE (1O86)",
 'AChE': "ACHE (1EVE)",
 'ADA': "ADA (1NDW)",
 'AMPC': "AMPC (1XGJ)",
 'AR': "AR (1XQ2)",
 'CDK2': "CDK2 (1CKP)",
 'COX2': "COX2 (5IKR)",
 'DHFR-MTX': "DHFR (3DFR)",
 'EGFR': "EGFR (1M17)",
 'ER': "ER (2P15)",
 'ER-agonist': "ER-agonist (2P15)",
 'FGFr1': "FGFR1 (1AGW)",
 'FXA': "FXA (1EZQ)",
 'GPB': "GPB (1A8I)",
 'GR': "GR (1M2Z)",
 'HIVPR': "HIVPR (1HPX)",
 'HIVRT': "HIVRT (1RT1)",
 'HSP90': "HSP90 (1UY6)",
 'InhA': "InhA (1P44)",
 'MR': "MR (2AA2)",
 'NA': "NA (1A4G)",
 'P38': "P38 (1KV2)",
 'PDE5': "PDE5 (1XP0)",
 'PPARg': "PPARg (1FM9\_D)",
 'PR': "PR (1SR7)",
 'SRC': "SRC (2SRC)",
 'Thrombin': "Thrombin (1BA8)",
 'Thymidinekinase': "TK (1KIM)",
 'Trypsin': "Trypsin (1BJU)",
 'VEGFr2': "VEGFr2 (1VR2)"}

In [28]:
# Make the publication table for Table 3
tf = partial(texify_val_error, force_sig=True, sig_digits=1)
print(enrichments.stack()
      .groupby(level=['level', 'method']).agg(['mean', 'std'])
      .unstack().T
      .groupby(axis=0, level='method').transform(tf)
      .xs('mean').drop(columns=['actives', 'decoys'], index=['mean_ens_dknn'])
      .rename(index=method_renamer)
      .to_latex(escape=False))

\begin{tabular}{llll}
\toprule
level &               0.01 &               0.05 &                0.1 \\
method        &                    &                    &                    \\
\midrule
eRF           &  \num{5.0 \pm 5.2} &  \num{2.0 \pm 1.7} &  \num{1.3 \pm 1.0} \\
RF            &  \num{5.4 \pm 5.3} &  \num{2.1 \pm 1.6} &  \num{1.4 \pm 0.9} \\
DNN           &  \num{4.0 \pm 4.2} &  \num{1.6 \pm 1.3} &  \num{1.2 \pm 0.9} \\
kNN           &  \num{1.5 \pm 3.1} &  \num{1.0 \pm 1.4} &  \num{0.9 \pm 0.9} \\
lSVR          &  \num{6.1 \pm 5.7} &  \num{1.9 \pm 1.6} &  \num{1.3 \pm 0.9} \\
SVR           &  \num{4.4 \pm 4.2} &  \num{1.7 \pm 1.4} &  \num{1.3 \pm 0.9} \\
LR            &  \num{6.2 \pm 5.9} &  \num{1.9 \pm 1.7} &  \num{1.0 \pm 1.0} \\
union         &  \num{7.3 \pm 5.9} &  \num{2.6 \pm 1.7} &  \num{1.7 \pm 0.9} \\
Union w/o kNN &  \num{7.5 \pm 5.6} &  \num{2.6 \pm 1.7} &  \num{1.7 \pm 1.0} \\
Mean          &  \num{5.4 \pm 4.8} &  \num{1.8 \pm 1.5} &  \num{1.2 \pm 0.9} \\
\bottomr

In [29]:
metadf = enrichments['lr'].unstack()[['actives', 'decoys']].groupby('system').first()
metadf

level,actives,decoys
system,,
ACE,48.0,1786.0
AChE,103.0,3848.0
ADA,39.0,920.0
AMPC,21.0,778.0
AR,79.0,2824.0
CDK2,70.0,2055.0
COX2,424.0,13098.0
DHFR-MTX,409.0,8216.0
EGFR,470.0,15849.0


In [30]:
percent_renamer = {"0.01": "1", "0.05": "5", "0.1": "10"}

ef_df = (enrichments
         .drop(columns=['mean_ens_dknn', "mean_ensemble", "union_dknn"])
         .drop(index=['actives', 'decoys'], level='level')
         .groupby(level=['system', 'level'])
# With error
         #.agg(['mean', 'std'])
         #.groupby(axis=1, level=0)
         #.transform(tf)
         #.xs('mean', level=1, axis=1)
# Alternative just rounding:
         .agg('mean')
         .round(1)
# Make the necessary adaptations
         .rename(index=percent_renamer, level='level')
         .rename(columns=method_renamer)
         .reset_index()         
        )

ef_df = ef_df.replace(system_rename)

for i, k in enumerate(dud_names):
    ef_df.iloc[3*i+1, 0] = "Active: {:.0f}".format(metadf.loc[k, 'actives'])
    ef_df.iloc[3*i+2, 0] = "Decoys: {:.0f}".format(metadf.loc[k, 'decoys'])


ef_df


method,system,level,eRF,RF,DNN,kNN,lSVR,SVR,LR,union
0,ACE (1O86),1,13.8,7.6,8.5,0.0,6.2,7.2,6.8,11.8
1,Active: 48,5,5.0,4.4,4.4,0.0,2.9,4.1,2.5,5.5
2,Decoys: 1786,10,3.0,2.8,3.1,0.2,1.5,2.7,1.4,3.1
3,ACHE (1EVE),1,5.9,7.8,4.0,0.9,6.8,5.4,6.9,9.6
4,Active: 103,5,3.1,3.3,1.9,1.8,2.6,2.4,2.2,2.7
5,Decoys: 3848,10,2.2,2.0,1.8,1.5,2.1,1.7,1.9,1.9
6,ADA (1NDW),1,3.0,4.9,3.6,0.3,8.2,2.5,8.2,8.3
7,Active: 39,5,1.5,1.6,2.0,1.0,1.6,1.4,1.6,1.7
8,Decoys: 920,10,0.8,0.8,1.6,0.7,1.2,1.4,1.1,1.0
9,AMPC (1XGJ),1,1.1,2.7,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
latex_string = ef_df.to_latex(longtable=False, escape=False, index=False)
print(latex_string)


\begin{tabular}{llrrrrrrrr}
\toprule
            system & level &   eRF &    RF &   DNN &   kNN &  lSVR &   SVR &    LR &  union \\
\midrule
        ACE (1O86) &     1 &  13.8 &   7.6 &   8.5 &   0.0 &   6.2 &   7.2 &   6.8 &   11.8 \\
        Active: 48 &     5 &   5.0 &   4.4 &   4.4 &   0.0 &   2.9 &   4.1 &   2.5 &    5.5 \\
      Decoys: 1786 &    10 &   3.0 &   2.8 &   3.1 &   0.2 &   1.5 &   2.7 &   1.4 &    3.1 \\
       ACHE (1EVE) &     1 &   5.9 &   7.8 &   4.0 &   0.9 &   6.8 &   5.4 &   6.9 &    9.6 \\
       Active: 103 &     5 &   3.1 &   3.3 &   1.9 &   1.8 &   2.6 &   2.4 &   2.2 &    2.7 \\
      Decoys: 3848 &    10 &   2.2 &   2.0 &   1.8 &   1.5 &   2.1 &   1.7 &   1.9 &    1.9 \\
        ADA (1NDW) &     1 &   3.0 &   4.9 &   3.6 &   0.3 &   8.2 &   2.5 &   8.2 &    8.3 \\
        Active: 39 &     5 &   1.5 &   1.6 &   2.0 &   1.0 &   1.6 &   1.4 &   1.6 &    1.7 \\
       Decoys: 920 &    10 &   0.8 &   0.8 &   1.6 &   0.7 &   1.2 &   1.4 &   1.1 &    1.0 \\
    

In [32]:
line_list = latex_string.split('\n')

In [33]:
for i, line in enumerate(line_list):
    print(i)
    print(line)

0
\begin{tabular}{llrrrrrrrr}
1
\toprule
2
            system & level &   eRF &    RF &   DNN &   kNN &  lSVR &   SVR &    LR &  union \\
3
\midrule
4
        ACE (1O86) &     1 &  13.8 &   7.6 &   8.5 &   0.0 &   6.2 &   7.2 &   6.8 &   11.8 \\
5
        Active: 48 &     5 &   5.0 &   4.4 &   4.4 &   0.0 &   2.9 &   4.1 &   2.5 &    5.5 \\
6
      Decoys: 1786 &    10 &   3.0 &   2.8 &   3.1 &   0.2 &   1.5 &   2.7 &   1.4 &    3.1 \\
7
       ACHE (1EVE) &     1 &   5.9 &   7.8 &   4.0 &   0.9 &   6.8 &   5.4 &   6.9 &    9.6 \\
8
       Active: 103 &     5 &   3.1 &   3.3 &   1.9 &   1.8 &   2.6 &   2.4 &   2.2 &    2.7 \\
9
      Decoys: 3848 &    10 &   2.2 &   2.0 &   1.8 &   1.5 &   2.1 &   1.7 &   1.9 &    1.9 \\
10
        ADA (1NDW) &     1 &   3.0 &   4.9 &   3.6 &   0.3 &   8.2 &   2.5 &   8.2 &    8.3 \\
11
        Active: 39 &     5 &   1.5 &   1.6 &   2.0 &   1.0 &   1.6 &   1.4 &   1.6 &    1.7 \\
12
       Decoys: 920 &    10 &   0.8 &   0.8 &   1.6 &   0.7 &   1.2 &  

In [34]:
line_list[4:-3]

['        ACE (1O86) &     1 &  13.8 &   7.6 &   8.5 &   0.0 &   6.2 &   7.2 &   6.8 &   11.8 \\\\',
 '        Active: 48 &     5 &   5.0 &   4.4 &   4.4 &   0.0 &   2.9 &   4.1 &   2.5 &    5.5 \\\\',
 '      Decoys: 1786 &    10 &   3.0 &   2.8 &   3.1 &   0.2 &   1.5 &   2.7 &   1.4 &    3.1 \\\\',
 '       ACHE (1EVE) &     1 &   5.9 &   7.8 &   4.0 &   0.9 &   6.8 &   5.4 &   6.9 &    9.6 \\\\',
 '       Active: 103 &     5 &   3.1 &   3.3 &   1.9 &   1.8 &   2.6 &   2.4 &   2.2 &    2.7 \\\\',
 '      Decoys: 3848 &    10 &   2.2 &   2.0 &   1.8 &   1.5 &   2.1 &   1.7 &   1.9 &    1.9 \\\\',
 '        ADA (1NDW) &     1 &   3.0 &   4.9 &   3.6 &   0.3 &   8.2 &   2.5 &   8.2 &    8.3 \\\\',
 '        Active: 39 &     5 &   1.5 &   1.6 &   2.0 &   1.0 &   1.6 &   1.4 &   1.6 &    1.7 \\\\',
 '       Decoys: 920 &    10 &   0.8 &   0.8 &   1.6 &   0.7 &   1.2 &   1.4 &   1.1 &    1.0 \\\\',
 '       AMPC (1XGJ) &     1 &   1.1 &   2.7 &   0.0 &   0.0 &   0.0 &   0.0 &   0.0 &    0

In [35]:
len(line_list[4:-3])

90

In [36]:
for i, line in enumerate(line_list[4:-3]):
    print(line)
    if (i+1) % 3 == 0:
        print('\\hline')

        ACE (1O86) &     1 &  13.8 &   7.6 &   8.5 &   0.0 &   6.2 &   7.2 &   6.8 &   11.8 \\
        Active: 48 &     5 &   5.0 &   4.4 &   4.4 &   0.0 &   2.9 &   4.1 &   2.5 &    5.5 \\
      Decoys: 1786 &    10 &   3.0 &   2.8 &   3.1 &   0.2 &   1.5 &   2.7 &   1.4 &    3.1 \\
\hline
       ACHE (1EVE) &     1 &   5.9 &   7.8 &   4.0 &   0.9 &   6.8 &   5.4 &   6.9 &    9.6 \\
       Active: 103 &     5 &   3.1 &   3.3 &   1.9 &   1.8 &   2.6 &   2.4 &   2.2 &    2.7 \\
      Decoys: 3848 &    10 &   2.2 &   2.0 &   1.8 &   1.5 &   2.1 &   1.7 &   1.9 &    1.9 \\
\hline
        ADA (1NDW) &     1 &   3.0 &   4.9 &   3.6 &   0.3 &   8.2 &   2.5 &   8.2 &    8.3 \\
        Active: 39 &     5 &   1.5 &   1.6 &   2.0 &   1.0 &   1.6 &   1.4 &   1.6 &    1.7 \\
       Decoys: 920 &    10 &   0.8 &   0.8 &   1.6 &   0.7 &   1.2 &   1.4 &   1.1 &    1.0 \\
\hline
       AMPC (1XGJ) &     1 &   1.1 &   2.7 &   0.0 &   0.0 &   0.0 &   0.0 &   0.0 &    0.0 \\
        Active: 21 &     5 & 